# Transactions — Overview

## Purpose
Understand database transactions and how they ensure data integrity through ACID properties. Transactions are fundamental to reliable database operations, ensuring that complex operations either complete fully or leave no trace.

## Key Questions
- What is a database transaction and why is it important?
- What are the ACID properties and how do they guarantee data reliability?
- How do `COMMIT` and `ROLLBACK` work in practice?
- What are common transaction isolation levels?

## Topics Covered
1. Transaction fundamentals
2. ACID properties explained
3. Commit and rollback operations
4. Transaction lifecycle visualization
5. Practical examples with SQLite

---
## What is a Transaction?

A **transaction** is a sequence of one or more database operations (INSERT, UPDATE, DELETE, SELECT) treated as a single logical unit of work. Either all operations succeed, or none of them take effect.

```
BEGIN TRANSACTION
    Operation 1: Debit $100 from Account A
    Operation 2: Credit $100 to Account B
COMMIT (or ROLLBACK if error)
```

---
## ACID Properties

ACID is an acronym representing four key properties that guarantee reliable transaction processing:

### 1. Atomicity ("All or Nothing")
- A transaction is treated as a single, indivisible unit
- If any part fails, the entire transaction is rolled back
- **Example**: Bank transfer — both debit and credit must succeed, or neither happens

### 2. Consistency ("Valid State Transitions")
- A transaction brings the database from one valid state to another
- All defined rules, constraints, and triggers are enforced
- **Example**: Account balance cannot go negative if that's a constraint

### 3. Isolation ("Concurrent Independence")
- Concurrent transactions execute as if they were sequential
- Intermediate states are not visible to other transactions
- **Example**: Two users updating the same record don't see partial changes

### 4. Durability ("Permanent Commitment")
- Once committed, changes are permanent even after system failures
- Data is written to non-volatile storage
- **Example**: After a successful commit, power loss won't lose the data

---
## ACID Properties Summary Table

| Property | Guarantee | Failure Scenario Protected |
|----------|-----------|---------------------------|
| **Atomicity** | All operations complete or none | Partial execution |
| **Consistency** | Database rules always enforced | Invalid data states |
| **Isolation** | Transactions don't interfere | Dirty reads, lost updates |
| **Durability** | Committed data survives crashes | Hardware/power failure |

---
## Practical Example: SQLite Transactions with Python

Let's demonstrate transaction behavior with `sqlite3`.

In [ ]:
import sqlite3

# Create an in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create accounts table
cursor.execute('''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        balance REAL NOT NULL CHECK(balance >= 0)
    )
''')

# Insert initial data
cursor.execute("INSERT INTO accounts (name, balance) VALUES ('Alice', 1000.00)")
cursor.execute("INSERT INTO accounts (name, balance) VALUES ('Bob', 500.00)")
conn.commit()

print("Initial balances:")
for row in cursor.execute("SELECT name, balance FROM accounts"):
    print(f"  {row[0]}: ${row[1]:.2f}")

In [ ]:
def transfer_funds(conn, from_account: str, to_account: str, amount: float):
    """Transfer funds between accounts with transaction handling."""
    cursor = conn.cursor()
    
    try:
        # Start transaction (implicit in sqlite3)
        print(f"\n🔄 Starting transfer: ${amount:.2f} from {from_account} to {to_account}")
        
        # Debit from source account
        cursor.execute(
            "UPDATE accounts SET balance = balance - ? WHERE name = ?",
            (amount, from_account)
        )
        print(f"  ➖ Debited ${amount:.2f} from {from_account}")
        
        # Credit to destination account
        cursor.execute(
            "UPDATE accounts SET balance = balance + ? WHERE name = ?",
            (amount, to_account)
        )
        print(f"  ➕ Credited ${amount:.2f} to {to_account}")
        
        # Commit the transaction
        conn.commit()
        print("  ✅ Transaction COMMITTED successfully!")
        return True
        
    except sqlite3.IntegrityError as e:
        # Rollback on constraint violation (e.g., negative balance)
        conn.rollback()
        print(f"  ❌ Transaction ROLLED BACK: {e}")
        return False
    except Exception as e:
        conn.rollback()
        print(f"  ❌ Transaction ROLLED BACK: {e}")
        return False

In [ ]:
# Successful transfer: Alice -> Bob $200
transfer_funds(conn, 'Alice', 'Bob', 200.00)

print("\nBalances after successful transfer:")
for row in cursor.execute("SELECT name, balance FROM accounts"):
    print(f"  {row[0]}: ${row[1]:.2f}")

In [ ]:
# Failed transfer: Bob tries to send more than he has
# This will trigger the CHECK constraint and rollback
transfer_funds(conn, 'Bob', 'Alice', 1000.00)

print("\nBalances after failed transfer (should be unchanged):")
for row in cursor.execute("SELECT name, balance FROM accounts"):
    print(f"  {row[0]}: ${row[1]:.2f}")

---
## Transaction Lifecycle Visualization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('Transaction Lifecycle', fontsize=18, fontweight='bold', pad=20)

# Define colors
colors = {
    'begin': '#3498db',
    'active': '#f39c12', 
    'commit': '#27ae60',
    'rollback': '#e74c3c',
    'end': '#9b59b6'
}

# Draw states
def draw_state(ax, x, y, label, color, width=2, height=1.2):
    box = FancyBboxPatch((x - width/2, y - height/2), width, height,
                         boxstyle="round,pad=0.05,rounding_size=0.2",
                         facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(box)
    ax.text(x, y, label, ha='center', va='center', fontsize=11, 
            fontweight='bold', color='white')

# States
draw_state(ax, 2, 7, 'BEGIN', colors['begin'])
draw_state(ax, 7, 7, 'ACTIVE\n(Operations)', colors['active'], width=2.5)
draw_state(ax, 12, 8.5, 'COMMIT', colors['commit'])
draw_state(ax, 12, 5.5, 'ROLLBACK', colors['rollback'])
draw_state(ax, 7, 2, 'END', colors['end'])

# Arrows
arrow_style = dict(arrowstyle='->', color='#2c3e50', lw=2, mutation_scale=15)

# BEGIN -> ACTIVE
ax.annotate('', xy=(5.5, 7), xytext=(3, 7), arrowprops=arrow_style)
ax.text(4.25, 7.4, 'Start', fontsize=9, ha='center')

# ACTIVE -> COMMIT
ax.annotate('', xy=(11, 8.5), xytext=(8.5, 7.3), arrowprops=arrow_style)
ax.text(10, 8.3, 'Success', fontsize=9, ha='center', color=colors['commit'])

# ACTIVE -> ROLLBACK  
ax.annotate('', xy=(11, 5.5), xytext=(8.5, 6.7), arrowprops=arrow_style)
ax.text(10, 5.7, 'Failure', fontsize=9, ha='center', color=colors['rollback'])

# COMMIT -> END
ax.annotate('', xy=(8, 2.3), xytext=(11.5, 8), arrowprops=arrow_style)
ax.text(10.5, 4.5, 'Changes\nPersisted', fontsize=9, ha='center')

# ROLLBACK -> END
ax.annotate('', xy=(7.5, 2.5), xytext=(11.5, 5), arrowprops=arrow_style)
ax.text(8.5, 3.5, 'Changes\nDiscarded', fontsize=9, ha='center')

# ACID properties box
acid_box = FancyBboxPatch((0.5, 0.5), 5, 4.5,
                          boxstyle="round,pad=0.1",
                          facecolor='#ecf0f1', edgecolor='#34495e', 
                          linewidth=2, linestyle='--')
ax.add_patch(acid_box)
ax.text(3, 4.5, 'ACID Guarantees', fontsize=12, fontweight='bold', ha='center')
ax.text(3, 3.8, '• Atomicity: All or nothing', fontsize=9, ha='center')
ax.text(3, 3.2, '• Consistency: Valid states only', fontsize=9, ha='center')
ax.text(3, 2.6, '• Isolation: No interference', fontsize=9, ha='center')
ax.text(3, 2.0, '• Durability: Permanent on commit', fontsize=9, ha='center')

plt.tight_layout()
plt.show()

---
## Transaction Isolation Levels

Isolation levels define how transactions interact with each other:

| Level | Dirty Read | Non-Repeatable Read | Phantom Read | Performance |
|-------|------------|---------------------|--------------|-------------|
| **READ UNCOMMITTED** | ✅ Possible | ✅ Possible | ✅ Possible | Fastest |
| **READ COMMITTED** | ❌ Prevented | ✅ Possible | ✅ Possible | Fast |
| **REPEATABLE READ** | ❌ Prevented | ❌ Prevented | ✅ Possible | Medium |
| **SERIALIZABLE** | ❌ Prevented | ❌ Prevented | ❌ Prevented | Slowest |

### Common Anomalies Explained:
- **Dirty Read**: Reading uncommitted changes from another transaction
- **Non-Repeatable Read**: Same query returns different results within a transaction
- **Phantom Read**: New rows appear in repeated queries

In [ ]:
# Setting isolation level in SQLite (default is SERIALIZABLE)
# SQLite supports: DEFERRED, IMMEDIATE, EXCLUSIVE

# Example: Using isolation_level parameter
conn_autocommit = sqlite3.connect(':memory:', isolation_level=None)  # Autocommit mode
conn_deferred = sqlite3.connect(':memory:', isolation_level='DEFERRED')  # Default
conn_immediate = sqlite3.connect(':memory:', isolation_level='IMMEDIATE')

print("SQLite Isolation Levels:")
print(f"  • None (Autocommit): Each statement auto-commits")
print(f"  • DEFERRED: Lock acquired on first write")
print(f"  • IMMEDIATE: Lock acquired at BEGIN")
print(f"  • EXCLUSIVE: Exclusive lock at BEGIN")

# Clean up
conn_autocommit.close()
conn_deferred.close()
conn_immediate.close()

---
## Best Practices for Transactions

### ✅ Do:
1. **Keep transactions short** — Hold locks for minimal time
2. **Use explicit transactions** — Don't rely on autocommit for multi-step operations
3. **Handle exceptions properly** — Always rollback on errors
4. **Use context managers** — Python's `with` statement ensures cleanup

### ❌ Don't:
1. **Hold transactions during user input** — Causes long-held locks
2. **Nest transactions incorrectly** — Understand savepoints if needed
3. **Ignore deadlocks** — Implement retry logic

In [ ]:
# Best Practice: Using context manager for transactions
import contextlib

@contextlib.contextmanager
def transaction(conn):
    """Context manager for database transactions."""
    try:
        yield conn.cursor()
        conn.commit()
        print("✅ Transaction committed")
    except Exception as e:
        conn.rollback()
        print(f"❌ Transaction rolled back: {e}")
        raise

# Usage example
conn_example = sqlite3.connect(':memory:')
conn_example.execute('CREATE TABLE test (id INTEGER, value TEXT)')

with transaction(conn_example) as cursor:
    cursor.execute("INSERT INTO test VALUES (1, 'Hello')")
    cursor.execute("INSERT INTO test VALUES (2, 'World')")

print("\nData in table:")
for row in conn_example.execute("SELECT * FROM test"):
    print(f"  {row}")

conn_example.close()

---
## 📌 Key Takeaways

1. **Transactions** group multiple operations into a single atomic unit of work

2. **ACID Properties** ensure reliable database operations:
   - **A**tomicity: All succeed or all fail
   - **C**onsistency: Database remains valid
   - **I**solation: Transactions don't interfere
   - **D**urability: Committed data persists

3. **COMMIT** makes changes permanent; **ROLLBACK** undoes uncommitted changes

4. **Isolation levels** balance consistency vs. performance — choose based on requirements

5. **Best practices**: Keep transactions short, use context managers, always handle errors

---

### Further Reading
- [SQLite Transaction Documentation](https://www.sqlite.org/lang_transaction.html)
- [PostgreSQL Transaction Isolation](https://www.postgresql.org/docs/current/transaction-iso.html)
- [Python DB-API 2.0 Specification](https://peps.python.org/pep-0249/)

In [ ]:
# Cleanup
conn.close()
print("Database connection closed.")